In [1]:
import scipy.stats as sps
import numpy as np
from matplotlib import pyplot as plt
from sklearn.decomposition import PCA, TruncatedSVD


%matplotlib inline

In [2]:
plt.rcParams['figure.figsize'] = 14, 8
plt.rcParams['font.family'] = 'DejaVu Serif'
plt.rcParams['lines.linewidth'] = 2
plt.rcParams['lines.markersize'] = 8
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12
plt.rcParams['legend.fontsize'] = 14
plt.rcParams['axes.titlesize'] = 24
plt.rcParams['axes.labelsize'] = 16

SEED = 42
np.random.seed(seed=SEED)

## Задача 2

**Класс для семплирования X, w, y из совместного распределения**

In [48]:
class JointDistribution(sps.rv_continuous):
    def __init__(self, alpha, sigma, n, 
                 x_seed=None, w_seed=None, y_seed=None):
        super().__init__(self)
        
        self.x_distr = sps.multivariate_normal(
            mean=np.zeros(n),
            cov=np.eye(n)*sigma,
            seed=x_seed
        )
        self.w_distr = sps.multivariate_normal(
            mean=np.zeros(n),
            cov=np.eye(n)/alpha,
            seed=w_seed
        )
        self.unif_distr = sps.uniform()
        self.unif_seed = y_seed
        self.n = n
    
    def _calc_y_proba(self, objects, weights):
        assert objects.ndim <= 2 and weights.ndim == 1
        
        if objects.ndim == 1:
            objects = objects.reshape(1, -1)
        
        return 1 / (1 + np.exp(-np.einsum("ij, j -> i", objects, weights)))
        
    def rvs(self, size):
        if not isinstance(size, int):
            raise ValueError(f"Expect type(size) == int, got {type(size)}")
        
        # objects.shape = (size, self.n)
        # weights.shape = (self.n)
        objects = self.x_distr.rvs(size)
        weights = self.w_distr.rvs(1)
        
        y_probas = self._calc_y_proba(objects, weights)
        bool_targets = self.unif_distr.rvs(size, self.unif_seed) >= 1 - y_probas
        targets = 2 * bool_targets - 1
        
        return objects, weights, targets
    
    def _pdf(self, obj, weight, trg):
        x_proba = self.x_distr.pdf(obj)
        w_proba = self.w_distr.pdf(weight)
        
        y_proba = self._calc_y_proba(obj, weight)
        y_proba = np.where(trg == 1, y_probas, 1 - y_probas)
        
        return x_proba * w_proba * y_proba

**Задаём константы**

In [68]:
n = 10
sigma = 8**2
alpha = 2**2
max_test_size = 10000
x_seed = SEED
w_seed = SEED+1
y_seed = SEED+2

distr = JointDistribution(alpha, sigma, n, x_seed, w_seed, y_seed)
x_test, w_true, y_test = distr.rvs(max_test_size)

**Вычисление AUC**

In [70]:
auc_arr = []

for test_size in (50, 500, 2000, 5000, max_test_size):
    x_data = x_test[:test_size]
    y_data = y_test[:test_size]
    
    pos_indices = np.flatnonzero(y_data == 1)
    pos_objects = x_data[pos_indices]
    neg_indices = np.flatnonzero(y_data == -1)
    neg_objects = x_data[neg_indices]
    
    k_total_pairs = pos_indices.size * neg_indices.size
    k_great_pairs = 0
    
    for pos_elem in pos_objects:
        for neg_elem in neg_objects:
            if np.dot(w_true, pos_elem - neg_elem) > 0:
                k_great_pairs += 1
                
    auc_arr.append(k_great_pairs / k_total_pairs)
    print(f"Finish to work with test_size = {test_size}")

Finish to work with test_size = 50
Finish to work with test_size = 500
Finish to work with test_size = 2000
Finish to work with test_size = 5000
Finish to work with test_size = 10000


In [74]:
print([round(auc, 4) for auc in auc_arr])

[0.9808, 0.9913, 0.9937, 0.9929, 0.9924]


## Задача 4

**Определение функций для эксперимента**  ($j^* = 0$)

In [46]:
def print_table_of_errors(true_conf_lvl, false_conf_lvl, 
                          true_hyp_p_values, false_hyp_p_values):
    TP = np.sum(true_hyp_p_values >= true_conf_lvl)
    TN = np.sum(false_hyp_p_values < false_conf_lvl)
    FN = np.sum(true_hyp_p_values < true_conf_lvl)
    FP = np.sum(false_hyp_p_values >= false_conf_lvl)
    
    format_str = '{:<16} | {:<12} | {:<12} | {:<8}'
    print(format_str.format(' ', 'H0 - верна', 'H0 - ложна', 'Всего'))
    print('-' * 57)
    print(format_str.format('# принятых H0', TP, FP, TP+FP))
    print('-' * 57)
    print(format_str.format('# отвергнутых H0', FN, TN, FN+TN))
    print('-' * 57)
    print(format_str.format('Всего', TP+FN, FP+TN, TP+FP+FN+TN))
    
    
def build_samples(m1, m2, n=100):
    stand_norm_distr = sps.multivariate_normal(
        mean=np.zeros(n),
        cov=np.eye(n),
        seed=SEED
    )

    sample = stand_norm_distr.rvs(m1+m2)
    sigma = np.sqrt(np.arange(1, 1+n)).reshape(1, -1)
    
    j_star = 0
    sample *= sigma
    sample[:m1, j_star] += 1
    
    return sample[:m1, :], sample[m1:, :]


def calc_t_stat(first_sample, second_sample):
    total_var = first_sample.var() * len(first_sample)
    total_var += second_sample.var() * len(second_sample)
    total_var /= (len(first_sample) + len(second_sample) - 2)
    
    coef = (1 / len(first_sample) + 1 / len(second_sample))**0.5
    mean_diff = first_sample.mean() - second_sample.mean()
    
    return mean_diff / coef / total_var**0.5

**Проверка гипотезы о равенстве средних**

In [56]:
alpha = 0.05
n = 100
m1 = 500
m2 = 500
t_stat_values = []
first_sample, second_sample = build_samples(m1, m2, n)

for i in range(n):
    x_sample, y_sample = first_sample[:, i], second_sample[:, i]
    t_stat_values.append(calc_t_stat(x_sample, y_sample))
    
t_stat_values = np.array(t_stat_values)
distr = sps.t(m1+m2-2)
first_pval = distr.cdf(t_stat_values)
second_pval = distr.cdf(-t_stat_values)
p_values = np.array([min(x, y) for x, y in zip(first_pval, second_pval)])

print_table_of_errors(alpha/2, alpha/2, p_values[1:], p_values[:1])

                 | H0 - верна   | H0 - ложна   | Всего   
---------------------------------------------------------
# принятых H0    | 93           | 0            | 93      
---------------------------------------------------------
# отвергнутых H0 | 6            | 1            | 7       
---------------------------------------------------------
Всего            | 99           | 1            | 100     


**Поправка Бенджамини-Хохберга**

In [57]:
def thr_line_equation(alpha, hyp_count):
    return np.linspace(alpha/hyp_count, alpha, hyp_count)


def get_adjusted_thr(p_values, alpha, hyp_count):
    sorted_p_values = np.sort(p_values)

    ticks = np.arange(hyp_count)
    thr_line = thr_line_equation(alpha, hyp_count)

    max_thr_ind = -1 if np.all(sorted_p_values >= thr_line) \
                    else np.flatnonzero(sorted_p_values < thr_line).max()

    adjusted_thr = 0 if max_thr_ind == -1 \
                    else thr_line[max_thr_ind]

    return adjusted_thr


adjusted_thr = get_adjusted_thr(p_values, alpha, n)
print(f"Adjusted threshold =", round(adjusted_thr, 4))
print()
print_table_of_errors(adjusted_thr, adjusted_thr, p_values[1:], p_values[:1])

Adjusted threshold = 0.0005

                 | H0 - верна   | H0 - ложна   | Всего   
---------------------------------------------------------
# принятых H0    | 99           | 0            | 99      
---------------------------------------------------------
# отвергнутых H0 | 0            | 1            | 1       
---------------------------------------------------------
Всего            | 99           | 1            | 100     


**Изучение зав-ти TP и FP от m1 и m2**

In [60]:
m1_arr = [10, 10, 50, 50, 100, 100, 250, 250]
m2_arr = [10, 25, 50, 100, 100, 250, 250, 500]
alpha = 0.05

for m1, m2 in zip(m1_arr, m2_arr):
    t_stat_values = []
    first_sample, second_sample = build_samples(m1, m2, n)

    for i in range(n):
        x_sample, y_sample = first_sample[:, i], second_sample[:, i]
        t_stat_values.append(calc_t_stat(x_sample, y_sample))

    p_values = 1 - sps.t(m1+m2-2).cdf(t_stat_values)
    print(f"m1 = {m1}, m2 = {m2}")
    print_table_of_errors(alpha, alpha, p_values[1:], p_values[:1])
    print("---------------------------------------------------------")

m1 = 10, m2 = 10
                 | H0 - верна   | H0 - ложна   | Всего   
---------------------------------------------------------
# принятых H0    | 95           | 1            | 96      
---------------------------------------------------------
# отвергнутых H0 | 4            | 0            | 4       
---------------------------------------------------------
Всего            | 99           | 1            | 100     
---------------------------------------------------------
m1 = 10, m2 = 25
                 | H0 - верна   | H0 - ложна   | Всего   
---------------------------------------------------------
# принятых H0    | 96           | 0            | 96      
---------------------------------------------------------
# отвергнутых H0 | 3            | 1            | 4       
---------------------------------------------------------
Всего            | 99           | 1            | 100     
---------------------------------------------------------
m1 = 50, m2 = 50
                 | H0

**Вывод:** Confusion matrix особо не меняется с ростом размера выборок